# Verification of the diagnosis by the RIBEIRO PRE-TRAINED Model 
  
- On this notebook we wiil look further into the stats of the RIBEIRO Neural Network and confirm the results annonced by his team.  
- We will work over **a part** of the giant dataset of more thant 2 millions 12-lead ECG.  
- To do so we will take 30000 ECG that will construct our validation dataset. It represents aproximately 1.3% of the whole training dataset
- We will create 7 datasets, one for each cardiac disease and the last one if the patient is healthy.  
- Also, in order to understand why the prediction is not correct, a {disease}_prob dataset will be created and some of the ECG that causes problem to the machine will be printed.


# I. Determine diseases datasets  
  
First we analyse the 1.5%_dataset to find what are the index of the abnormal-ECG, we categorise them into 7 lists.  
Those lists will be limited to two thousands indexes, it means that our stats won't be perfect but they will approximate the real ones.


  ### I.1. Open datas and take only 30000 samples


Taking the annotated ECG in order to compare the prediction to the annotation at the end.  
We want **4 or 5 hundreds ECG of each diseases**, then 30000 datas looks alright to do so.

In [4]:
import numpy as np
import h5py
from tqdm import tqdm

TEST_SHAPE = 30000 
PATH_TO_GIANT = '/media/cotxetjordi/TOSHIBA EXT/Data_2/'

with h5py.File(PATH_TO_GIANT + "preprocessed_traces15pc.hdf5", "r") as f:
    i = 0
    signals = []
    id_exams = []
    ids = list(f["id_exam"][:60000])  #Only 40000 raw data needed

    for index,data in tqdm(enumerate(f["signal"])):

        id = ids[index]

        if id < 2200000: # Only takes exams with annotations available in annotations.csv
            
            signals.append(data)
            id_exams.append(id)
            i+=1
            
        if i == TEST_SHAPE:
            break
            
        
signals = np.array(signals)
id_exam = np.array(id_exams)
print("La taille du dataset est de :", np.shape(signals))
print(len(id_exams))


40635it [00:07, 5340.05it/s]


La taille du dataset est de : (30000, 4096, 12)
30000


### I.2.a) Save the datas to use it later 


In [6]:
with h5py.File(PATH_TO_GIANT + 'datas_to_verif.hdf5', 'a') as new_file:
    id_exam_data = new_file.create_dataset(name = "id_exam",data = id_exams)
    signals_data = new_file.create_dataset(name = "signal",data= = signals)
    

### I.2.b) Load them

In [1]:
import h5py
PATH_TO_GIANT = 'F:/Data_2/'

with h5py.File(PATH_TO_GIANT + 'datas_to_verif.hdf5', 'r') as new_file:
    id_exams = list(new_file["id_exam"][()])
    X = new_file["signal"][()]



## I.3. Create 7 datasets, one for each disease



In [1]:
import time
import csv
PATH_TO = 'F:/Data_2/' #Path to your csv file with only the columns
counter = -1
start_time = time.time()
with open(PATH_TO + 'annotations_processed.csv','r', newline='') as csvfile:
        COUNTER = 0
        i = 1
        carac = {"np":[],"1dAVB":[],"RBBB":[],"LBBB":[],"SB":[],"AF":[],"ST":[]}
        f = csv.reader(csvfile)
        indexes = list(carac.keys())
        
        
        for line in f:
            if i != 1:
                id_ex = int(line[0])

                
            if id_ex in id_exams:

                if "1" in line and line.count("1") == 1 : #Just taking the "mono-disease" ECG to analyse
                    disease_ids = carac[indexes[line.index("1")]]

                    if len(disease_ids) < 2000:
                        disease_ids.append(id_exams.index(id_ex))

                        COUNTER +=1

          
               
                elif len(carac["np"]) < 2000:
                    carac["np"].append(id_exams.index(id_ex))
                    COUNTER +=1
       
            if i % 50000 == 0:
                    print(COUNTER)
                    print("--- %s seconds ---" % (time.time() - start_time))     
                    
            if COUNTER == 5500:
                break
                
            i += 1

           
print("--- %s seconds ---" % (time.time() - start_time)) 


NameError: name 'id_ex' is not defined

### I.3.a)bis Create the preprocessed csv annotations


In [15]:
import csv
with open('F:/Data_2/annotations.csv','r', newline='') as csvfile:
    with open('F:/Data_2/annotations_processed.csv','w',newline='') as fichiercsv:
        writer=csv.writer(fichiercsv)
        i = 0
        read = csv.reader(csvfile)
        for line in read:


                i+=1
                writer.writerow([line[0]]+line[4:10]) # Only useful datas for testing


### I.3.b) Save the datas

In [33]:
import numpy as np

for index, (dis,ids) in enumerate(carac.items()) :

    np.save(dis,ids)


### I.3.c) Load the 7 datasets

In [2]:
import numpy as np
path_to_dis = "F:/Data_2/diseases/"

carac = {"np":[],"1dAVB":[],"RBBB":[],"LBBB":[],"SB":[],"AF":[],"ST":[]}
for index,(key,ids) in enumerate(carac.items()):
    
    carac[key] = np.load(path_to_dis + key + ".npy")

    print(key,"has", len(carac[key]),"samples")
    



np has 2000 samples
1dAVB has 365 samples
RBBB has 605 samples
LBBB has 392 samples
SB has 416 samples
AF has 466 samples
ST has 610 samples


### I.4 Create an array composed by 7 arrays of disease-acquisition

In [3]:
X_test_all = np.array([[X[index] for index in ids] for ids in carac.values()]) #don't forget to convert each of the 7 lists when you will predict using the pre-trained model

    

## II. Prediction test
### II.1 Load and compile the model

In [4]:

# %% Import packages
import tensorflow.compat.v1 as tf  #Fait appel à la binliothèque Tensorflow Version 1.xx
tf.disable_v2_behavior()
import numpy as np
import warnings
import argparse

warnings.filterwarnings("ignore")
import keras
from keras.models import load_model
from keras.optimizers import Adam
import h5py

PATH_TO_MODEL  = 'C:/Users/valer/Desktop/Stage_Notebook/automatic-ecg-diagnosis-master/model/'



# Import model
model = keras.models.load_model(PATH_TO_MODEL+ 'model.hdf5',compile=False)

#model = load_model(PATH_TO_MODEL, compile=False)
model.compile(loss='binary_crossentropy', optimizer=Adam(),metrics = [tf.keras.metrics.CategoricalAccuracy()])


print("Model saved")

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Using TensorFlow backend.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model saved


### II.2. Prediction test
### II.2.a) Auto-evaluation
If we use the Keras-pre-implemented function **"evaluate"** we can see that the prediction score is not as good as announced. That is why we will **manually** determine the tresholds and practice our tests.

In [97]:
results = []
y_true = [[[0]*6 for i in range(len(carac["np"] ) ) ] , [[1,0,0,0,0,0]for i in range(len(carac["1dAVB"] ) ) ] , [[0,1,0,0,0,0]for i in range(len(carac["RBBB"] ) ) ] , [[0,0,1,0,0,0]for i in range(len(carac["LBBB"] ) ) ], [[0,0,0,1,0,0]for i in range(len(carac["SB"] ) ) ] , [[1,0,0,0,1,0]for i in range(len(carac["AF"] ) ) ] , [[1,0,0,0,0,1]for i in range(len(carac["ST"] ) ) ] ]
for ind,X in enumerate(X_test_all): 
    results.append(model.evaluate(np.array(X), np.array(y_true[ind]), batch_size=128,verbose = 1))
print([i[1] for i in results])

610/610 [==============================] - 2s 3ms/step
[0.5453733801841736, 0.5464203953742981, 0.5529597997665405, 0.5596546530723572, 0.5647485852241516, 0.5611258149147034, 0.5523701906204224]


### II.2.b)i. Prediction over our mono-disease array

In [9]:
from scipy.special import softmax
y_score = []
lens = [len(carac[dise]) for dise in carac.keys()] #the number of samples for each disease
y_true = [[[0]*6 for i in range(lens[0] )  ] + [[1,0,0,0,0,0]for i in range(lens[1] ) ] + [[0,1,0,0,0,0]for i in range(lens[2] )  ] + [[0,0,1,0,0,0]for i in range(lens[3] ) ]+ [[0,0,0,1,0,0]for i in range(lens[4] ) ] + [[0,0,0,0,1,0]for i in range(lens[5] ) ] + [[0,0,0,0,0,1]for i in range(lens[6]) ] ]
for X in X_test_all:    
    y_score += list(model.predict(np.array(X),verbose =1))
print("Prediction saved!")

610/610 [==============================] - 2s 3ms/step
Prediction saved!


### II.2.b)ii. Get optimal precision and recall using the best thresholds
<span style="color:red"> Here we have a problem with the **first disease** we can see that the machine precision is really bad compared to others.  
So, let's determine the precision by ourselves. </span>

In [10]:
from sklearn.metrics import (confusion_matrix,
                             precision_score, recall_score, f1_score,
                             precision_recall_curve, average_precision_score)
y_true = np.array(y_true[0])
y_score = np.array(y_score)
def get_optimal_precision_recall(y_true, y_score):
    """Find precision and recall values that maximize f1 score."""
    n = np.shape(y_true)[1]
    opt_precision = []
    opt_recall = []
    opt_threshold = []
    for k in range(n):
        # Get precision-recall curve
        precision, recall, threshold = precision_recall_curve(y_true[:, k], y_score[:, k])
        # Compute f1 score for each point (use nan_to_num to avoid nans messing up the results)
        f1_score = np.nan_to_num(2 * precision * recall / (precision + recall))
        # Select threshold that maximize f1 score
        index = np.argmax(f1_score)
        opt_precision.append(precision[index])
        opt_recall.append(recall[index])
        t = threshold[index-1] if index != 0 else threshold[0]-1e-10
        opt_threshold.append(t)

    return np.array(opt_precision), np.array(opt_recall), np.array(opt_threshold)

p,r,t = get_optimal_precision_recall(y_true,y_score)
print("precision:",p)
print("recall:",r)
print('thresholds:',t)

precision: [0.67234043 0.92834395 0.86774942 0.8800905  0.92393736 0.92868217]
recall: [0.86575342 0.96363636 0.95408163 0.93509615 0.88626609 0.98196721]
thresholds: [0.17756437 0.22543368 0.2129037  0.19892462 0.32807842 0.15627445]


### II.2.c) Convert to one hot encoding
We chose our threshold adaptated to this dataset, then we convert our **raw prediction into one hot encoding predictions** in order to compare it to the annotations.

In [38]:
import tqdm
y_score_conv = []
thresholds = t


for pred in tqdm.tqdm(y_score):
    res = [0,0,0,0,0,0]
    for ind,val in enumerate(pred):

        if val >= thresholds[ind]:
           
            res[ind] = 1 

    y_score_conv.append(res)


100%|██████████| 4854/4854 [00:00<00:00, 146956.10it/s]


### II.3. Determine how much predictions errors are made and what are the ECGs that cause those errors

In [55]:
lens_ok = []
for i,len_ in enumerate(lens):
    if i == 0:
        lens_ok.append(len_)
    else: 
        lens_ok.append(lens_ok[i-1] + lens[i])

pos = 0
failed_prec = []
sensi =  0
failed_sensi = []
nb_fail = 0

for last_samp in lens_ok:
        failed_dis = []
        failed_sensi_dis = []
        for i in range(pos,last_samp):
            if not (y_true[i] == y_score_conv[i]).all(): 
                failed_dis.append(i)
                nb_fail += 1

                if i > lens_ok[0]: #Only look at the prediction with disease
                    try:
                        index_dis = list(y_true[i]).index(1)
                        if y_score_conv[i][index_dis] != 1:
                            sensi += 1
                            failed_sensi_dis.append(i) 
                            
                    except ValueError:#If no disease is predicted
                        sensi += 1
                        failed_sensi_dis.append(i) 
                    
        failed_sensi.append(failed_sensi_dis)        
        failed_prec.append(failed_dis)
        pos = i 


[]


### II.4. Create precision datas

In [63]:
print("------------------------------/TOTAL\---------------------------------")
print("Over a number of 4900 samples the prediction failed:",nb_fail,"times")
print("It means the total precision is {0:.3g} %".format((1-(nb_fail/(pos+1)))*100))
print("--/AND\--")
print("Over a number of 4900 samples the prediction failed to detect a disease:",sensi,"times")
print("It means the total sensitivity is {0:.3g} %".format((1-(sensi/(pos+1)))*100))
print("------------------------------------------------------------------")

for ind,fails in enumerate(failed_prec):
    print("-------------------------------/{}\-------------------------------".format(list(carac.keys())[ind]))
    print("Over a number of {} samples of {} ECG the prediction failed to detect a disease: {} times".format(lens[ind],list(carac.keys())[ind],len(fails)))
    print("It means the precision is {0:.3g} %".format((1-(len(fails)/lens[ind]))*100))
    print('Examples of errors: {} instead of {}'.format([(i,y_score_conv[i]) for i in fails[:10]],y_true[lens_ok[ind]-1]))
    print("--/AND\--")
    print("Over a number of {} samples of {} ECG the prediction failed to detect a disease: {} times".format(lens[ind],list(carac.keys())[ind],len(failed_sensi[ind])))
    print("It means the sensitivity is {0:.3g} %".format((1-(len(failed_sensi[ind])/lens[ind]))*100))
    print("------------------------------------------------------------------")

------------------------------/TOTAL\---------------------------------
Over a number of 4900 samples the prediction failed: 495 times
It means the total precision is 89.8 %
--/AND\--
Over a number of 4900 samples the prediction failed to detect a disease: 181 times
It means the total sensitivity is 96.3 %
------------------------------------------------------------------
-------------------------------/np\-------------------------------
Over a number of 2000 samples of np ECG the prediction failed to detect a disease: 161 times
It means the precision is 92 %
Examples of errors: [(8, [0, 0, 0, 1, 0, 0]), (9, [1, 0, 0, 0, 0, 0]), (13, [1, 0, 1, 0, 0, 0]), (19, [0, 0, 0, 1, 0, 0]), (22, [0, 0, 0, 0, 1, 0]), (43, [1, 0, 0, 0, 0, 0]), (47, [0, 0, 1, 0, 0, 1]), (48, [1, 0, 0, 1, 0, 0]), (72, [1, 0, 0, 0, 0, 0]), (86, [0, 0, 0, 0, 0, 1])] instead of [0 0 0 0 0 0]
--/AND\--
Over a number of 2000 samples of np ECG the prediction failed to detect a disease: 0 times
It means the sensitivity is 10